In [1]:
import requests
import re
import pandas as pd
import folium
import numpy as np
from bs4 import BeautifulSoup
from geopy.geocoders import Nominatim

In [2]:
ciudades_natales = {'ciudad': [], "pais": []}

In [3]:
art = pd.read_csv('../data/artists.csv', index_col = 0)
art.head(2)

,name,years,genre,nationality,bio,wikipedia,paintings
id,,,,,,,
0,Amedeo Modigliani,1884 - 1920,Expressionism,Italian,Amedeo Clemente Modigliani (Italian pronunciat...,http://en.wikipedia.org/wiki/Amedeo_Modigliani,193
1,Vasiliy Kandinskiy,1866 - 1944,"Expressionism,Abstractionism",Russian,Wassily Wassilyevich Kandinsky (Russian: Васи́...,http://en.wikipedia.org/wiki/Wassily_Kandinsky,88


In [4]:
links = art['wikipedia']

In [9]:
for url in links:
    try:
        html = requests.get(url)
        soup = BeautifulSoup(html.content, "html.parser")
        productos = soup.find_all("div",{"class": "birthplace"})
        x = productos[0].find_all('a')

        if len(x) == 2: 
            ciudades_natales['ciudad'].append(x[0].get('title')) 
            ciudades_natales['pais'].append(x[1].get('title'))
        else:
            ciudades_natales['ciudad'].append(x[0].get('title')) 
            ciudades_natales['pais'].append('Información no disponible.')  
    except:
        ciudades_natales['ciudad'].append('Información no disponible.')
        ciudades_natales['pais'].append('Información no disponible.')


In [10]:
lugar_nacimiento = pd.DataFrame(ciudades_natales)
lugar_nacimiento

,ciudad,pais
0,Livorno,Kingdom of Italy
1,Moscow,Russian Empire
2,Guanajuato City,Información no disponible.
3,Paris,Información no disponible.
4,Lessines,Información no disponible.
...,...,...
95,Pittsburgh,Pennsylvania
96,French Second Republic,Información no disponible.
97,Urbino,Duchy of Urbino
98,Caprese Michelangelo,Información no disponible.


In [11]:
ciudades = ciudades_natales['ciudad']

In [12]:
def sacar_coord(x):
    try: 
        geolocator = Nominatim(user_agent="MyApp")
        location = geolocator.geocode(x)
        return location.latitude, location.longitude
    except:
        return 'Sin información'

In [13]:
geo_ciudad = pd.DataFrame(lugar_nacimiento["ciudad"].apply(sacar_coord))

In [28]:
geo_ciudad.head(5)

,ciudad,latitud,longitud
0,"(42.790216599999994, 10.340278678735025)",42.790217,10.340279
1,"(55.7504461, 37.6174943)",55.750446,37.617494
2,"(19.3834636, -99.2190861)",19.383464,-99.219086
3,"(48.8588897, 2.3200410217200766)",48.858890,2.320041
4,"(50.7123036, 3.8301131)",50.712304,3.830113


In [16]:
def sacar_latitud(x):
    try:
        return float(x[0])
    except:
        return np.nan

In [17]:
geo_ciudad["latitud"] = geo_ciudad["ciudad"].apply(sacar_latitud) 

In [18]:
def sacar_longitud(x):
    try:
        return float(x[1])
    except:
        return np.nan

In [22]:
geo_ciudad["longitud"] = geo_ciudad["ciudad"].apply(sacar_longitud) 

In [41]:
geo_def = geo_ciudad.dropna()

In [42]:
mapa=folium.Map(location=[40, 15],
                tiles='openstreetmap',
                zoom_start=4)

for index, row in geo_def.iterrows():
    folium.Marker([row["latitud"], row['longitud']],
                  radius=10,
                  icon=folium.Icon(icon = "cloud", color = "red")).add_to(mapa)

mapa